# How to use Equity prices to estimate default probabilities?


### In 1974, Merton published "On the Pricing of Corporate Debt: The Risk Structure of Interest Rates", in which he proposes a model where a company’s equity is an option on the assets of the company


$V_0$ : Value of company’s assets today\
$V_T$ : Value of company’s assets at time T\
$E_0$ : Value of company’s equity today\
$E_T$ : Value of company’s equity at time T\
$D$: Debt repayment due at time T\
$\sigma_V$ : Volatility of assets (assumed constant)\
$\sigma_E$ : Instantaneous volatility of equity

If $V_T$ < $D$, it is rational for the company to default on the debt at time $T$.The value of the equity $E_T = 0$\
If $V_T$ > $D$, the company should make the debt repayment at time $T$ and the value of the equity $E_T = V_T- D$\
Therefore, the value of the firm’s equity at time T is $E_T = max(V_T-D; 0)$

### The Black–Scholes–Merton formula gives the value of the equity today as
### (1) $$E_0 = V_0 N(d_1)-De^{rT}N(d_2)$$

$$d_1 = \frac{ln\left(V_0/D\right)+\left(r+\sigma_V^2/2\right)T}{\sigma_V\sqrt{T}}$$

$$d_2 = d_1 - \sigma_V \sqrt{T}$$

### The risk-neutral probability that the company will default on the debt is 
$$1-N(d_2) = N(-d_2)$$\
To calculate this, we require $V_0$ and $\sigma_V$ We only have one equation for two unknowns, we need a second equation to estimate $V_0$ and $\sigma_V$ 

### From Ito’s lemma we have, 
### (2) $$\sigma_E E_0 = \frac{\partial{E}}{\partial{V}}\sigma_V V_0 = N(d_1)\sigma_V V_0$$

In [88]:
import numpy as np
import scipy.stats as si
from scipy import optimize

In [64]:
# First Equation, the B&S call price

def BS_call(V0, D, T, r, sigma):
    
    #V0: value of company's asset today
    #D: debt repayment due at time T
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of assets (assumed constant)
    
    d1 = (np.log(V0 / D) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(V0 / D) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (V0 * si.norm.cdf(d1, 0.0, 1.0) - D * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    # cdf means Cumulative distribution function
    
    return call

In [56]:
#Second Equation, from Ito's Lemma

def volatility_of_equity(V0, D, T, r, sigma, E0):
    
    #V0: value of company's asset today
    #D: debt repayment due at time T
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of assets (assumed constant)
    #E0: value of company's equity today
    
    d1 = (np.log(V0 / D) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    volatility = si.norm.cdf(d1, 0.0, 1.0)*sigma*V0*(1/E0)
    # cdf means Cumulative distribution function
    
    return volatility 

In [57]:
BS_call(12400000, 10000000, 1, 0.05, 0.2123)

In [108]:
volatility_of_equity(12400000, 10000000, 1, 0.05, 0.2123, 3000000)

### In order to find $V_0$ and $\sigma_V$, we define the following functions: 
$$F(V_0,\sigma_V)= V_0 N(d_1)-De^{rT}N(d_2)-E_0$$
$$G(V_0,\sigma_V)= N(d_1)\sigma_V V_0 \frac{1}{E_0}-\sigma_E$$

### Our goal is to find the vector $(V_0,\sigma_V)$ such as:
$$F(V_0,\sigma_V) = 0$$
$$G(V_0,\sigma_V) = 0$$
### That means we need to find the roots of these two functions, but the problem is that there is no closed form solution to this equation. We must vary the $(V_0,\sigma_V)$ vector with a trial and error process in order to optimize its value and find the roots. Fortunately, many multi-objective optimisation programms already exist in the scipy.optimize library.

In [175]:
def fonction_a_optimiser(X):
    
    V0 = X[0]
    sigma_asset = X[1] 
    
    if not 0<=sigma_asset<=1 :   
        return [np.nan, np.nan]
    if not 0<=sigma_asset :
         return [np.nan, np.nan]
    
    E0 = 3004198.18
    D = 10000000
    T = 1
    r = 0.05 
    sigma_equity = 0.8005288044340673
    
    a = BS_call(V0, D, T, r, sigma_asset)-E0
    
    b = volatility_of_equity(V0, D, T, r, sigma_asset, E0)-sigma_equity
    
    return [a, b]

In [176]:
fonction_a_optimiser([12400000, 0.2123])

### The purpose of HYBRD is to find a zero of a system of N non-linear functions in N variables by a modification of the Powell hybrid method.  The user must provide a subroutine which calculates the functions.  The Jacobian is then calculated by a for-ward-difference approximation.

In [186]:
sol = scipy.optimize.root(fonction_a_optimiser, x0=[1000_000,0.5],method='hybr')
print(sol)

    fjac: array([[-1.00000000e+00, -1.65536732e-06],
       [ 1.65536732e-06, -1.00000000e+00]])
     fun: array([-3.00419717e+06, -8.00527066e-01])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 15
     qtf: array([ 3.00419717e+06, -4.17252275e+00])
       r: array([nan, nan, nan])
  status: 5
 success: False
       x: array([1.e+06, 5.e-01])


In [188]:
sol.qtf

array([ 3.00419717e+06, -4.17252275e+00])

### We get a really good approximation for the value of the company's assets $V_0$. However the HYBRYD method do not accept the constraint on the volatility of assets: $0\leq  \sigma_V \leq 1$. This is a huge problem since we get completely out of the range values for $\sigma_V$, even with an initilal guess at $0.5$. How can we resolve this issue?

### One solution could be to find a bijective function $f$, such as  $f:[0,1]\longmapsto R^2$
### apply $f$ to $\sigma_V$ $\Longrightarrow$ find the roots with HYBRD method on $f(\sigma_V)$ $\Longrightarrow$ apply $f^{-1}$
### Inverse Hyperbolic Tangent is a good candidate since it satisfies all the conditions above 

In [197]:
E0 = 3004198.18
D = 10000000
T = 1
r = 0.05 
sigma_equity = 0.8005288044340673

def fonction_arctanh(X):
    
    V0 = X[0]
    sigma_asset = X[1] 
    
    new_sigma = np.arctanh(2*sigma_asset-1)
    
    a = BS_call(V0, D, T, r, new_sigma)-E0
    
    b = volatility_of_equity(V0, D, T, r, new_sigma, E0)-sigma_equity
    
    return [a, b]

In [199]:
sol = scipy.optimize.root(fonction_arctanh, x0=[1000_000,0.5],method='hybr')
print(sol)

    fjac: array([[1., 0.],
       [0., 1.]])
     fun: array([-3.00419818e+06, -8.00528804e-01])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 15
     qtf: array([-3.00419818e+06, -8.00528804e-01])
       r: array([nan, nan, nan])
  status: 5
 success: False
       x: array([1.e+06, 5.e-01])


<ipython-input-64-2e0661643432>:11: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(V0 / D) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
<ipython-input-64-2e0661643432>:12: RuntimeWarning: divide by zero encountered in double_scalars
  d2 = (np.log(V0 / D) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
<ipython-input-197-5f03f05edcbd>:12: RuntimeWarning: invalid value encountered in arctanh
  new_sigma = np.arctanh(2*sigma_asset-1)


In [200]:
scipy.optimize.root(fonction_a_optimiser, x0=[1000_000,0.5], method='df-sane')

     fun: array([-3.00419717e+06, -8.00527066e-01])
 message: 'too many function evaluations required'
    nfev: 1000
     nit: 0
 success: False
       x: array([1.e+06, 5.e-01])

In [185]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

x = np.arange(start=1, stop=50_000_000, step=10_000)
y = np.arange(start=0, stop=1, step=0.0002)
X,Y = np.meshgrid(x,y)
Z = fonction_a_optimiser([x,y])


fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')


# Plot a 3D surface
ax.plot_surface(X, Y, Z)


plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()